In [2]:
import torch
import numpy as np

In [3]:
# 加载数据
bikes_numpy = np.loadtxt('../data/p1ch4/bike-sharing-dataset/hour-fixed.csv',
                         dtype=np.float32, 
                         delimiter=",",
                         skiprows=1,
                        # 将日期字符串转换为与第1列中的月和日对应的数字
                         converters={1: lambda x: float(x[8:])})
bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

我们可能想要把 2 年的数据集分成更细的观察周期，如按天划分，这样我们就有了序列长度为 L、样本数量为 N 的集合 C。换句话说，我们的时间序列数据集将是一个维度为 3、形状为N×C×L 的张量。 C 依然是 17，而 L 则是 24，表示一天中的 24 小时。

上面的数据中，第 1 列是索引，它按全局数据排序；第 2 列是日期；第 6 列是一天的时间。现在我们已经有了创建一个每日骑行次数序列以及其他外生变量数据集所需的一切。我们的数据集已排好序，但如果没有排序，可以使用 torch.sort()来对数据集进行排序。

为了获得每日小时数据集，我们所要做的就是以 24 小时为单位来查看同一个张量。让我们看看 bikes 张量的形状和步长：

In [4]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

它有 17520 小时， 17 列。现在我们重新调整数据，让它有 3 个轴，即日、小时，然后是17 列：

In [5]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

首先， bikes.shape[1]是 17，即 bikes 张量的列的数量。但这段代码的关键是对 view()的调用，这非常重要：它会改变张量查看存储的相同数据的方式。

在一个张量上调用 view()会返回一个新的张量，该张量会在不改变存储的情况下改变维度和步长信息。这意味着我们可以以 0 代价重新排列张量，因为没有数据会被复制。对 view()的调用要求我们为返回的张量提供新的形状。我们使用−1 作为占位符，占位符的数量反映了在给定其他维度和原始元素数量的情况下，还剩下多少索引。

对于张量 daily_bikes 来说，其步长告诉我们沿着小时维度（第 2 个维度）前进 1 需要在存储（或一组列）中前进 17 个位置，而沿着日维度（第 1 个维度）前进需要我们前进一定数量的元素，这个长度等于存储中的一行的长度乘 24，这里长度为 408，即 17×24。

我们可以看到，最右边的维度是原始数据集中的列数。然后，中间的维度为时间，并将其分割成连续的 24 小时。换句话说，对于 C 个通道，有 N 个序列，每天 L 小时。为得到我们想要的N × C × L 次序，我们需要转置张量：

In [6]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride() 

(torch.Size([730, 17, 24]), (408, 1, 17))

为了更容易地呈现数据，我们暂时只关注第一天的数据。我们初始化一个零填充矩阵，其行数等于一天中的小时数，列数等于天气状况级别数。有 4 个级别： 1 表示晴天， 4 表示大雨/大雪。

In [7]:
first_day = bikes[:24].long()
# first_day.shape # [24, 17]
weather_onehot = torch.zeros(first_day.shape[0], 4)
first_day[:,9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

然后根据每行对应的级别将1 散置到矩阵中。

In [8]:
weather_onehot.scatter_(dim=1, 
                        # 减1是由于天气状况的级别是1~4，而索引是从0开始的
                        index = first_day[:,9].unsqueeze(1).long() - 1,
                        value=1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

最后，我们使用 cat()函数将矩阵连接到原始数据集。让我们看看第 1 个结果：

In [9]:
torch.cat((bikes[:24], weather_onehot), 1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

在这里，我们指定了原始的张量 bikes，以及沿着列（即 1）连接的独热编码的“天气状况”矩阵。换句话说，将 2 个数据集的列堆叠在一起，或者将新的独热编码列追加到原始数据集。为了使 cat()执行成功，张量必须在其他维度上具有相同的大小，对于本例即为行维度。请注意，得到的新的最后 4 列的值是 1、 0、 0、 0，这与天气状况值为 1 时的结果完全一样。

我们也可以对重塑的张量 daily_bikes 做同样的操作，它的形状记为(B, C, L)，其中 L = 24。我们首先创建一个 0 张量，其具有相同的 B 和 L，但是附加列数为 C：

In [10]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])
daily_weather_onehot.shape

torch.Size([730, 4, 24])

然后我们将独热编码分散到 C 维张量中，因为这个操作是在适当的位置进行的，所有只有张量的内容会改变：

In [11]:
daily_weather_onehot.scatter_(1, daily_bikes[:, 9, :].long().unsqueeze(1) - 1, 1.0)
daily_weather_onehot.shape

torch.Size([730, 4, 24])

我们沿着 C 维进行连接：

In [12]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), 1)

转换变量， 使其从 0.0 变为 1.0。

In [17]:
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] - 1.0) / 3.0

对变量进行重新调整有多种可能。我们可以将它们的范围映射到[0.0,1.0]：

In [18]:
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min) / (temp_max - temp_min))

或者减去均值再除以标准差：

In [19]:
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - torch.mean(temp)) / torch.std(temp))